# Objetivo do código

Esse códido é o início de uma possível implementação do Chatbot. Foi pensando em treinar o modelo para ele classificar um nível de cada vez, usufruindo da resposta de um nível para treinar o próximo. Dessa maneira, o único input semlehante para o treinamento de todos os níveis é a descrição da compra. Até agora foi feito apenas o treinamento do primeiro nível

# Importação das bibliotecas necessárias

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0477611df23da0d0dfc4eed0a79e325f94bfe4bf03de0fcb35bd99613bfedfdf
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
# Instalando os pacotes necessários
from google.colab import drive
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

import matplotlib.pyplot as plt
import math
import spacy
from sentence_transformers import SentenceTransformer

import gensim
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
categories = ['Energy & Utilities', 'Human Resources', 'Logistics', 'Manufacturing',
              'Professional Services', 'R&D Equipment', 'Real Estate & Facilities',
              'Sales, Marketing & Events', 'Technology/Telecom', 'Travel & Expense']

subcategories_per_category = {}

for category in categories:
    subcategories = df[df['Level 1'] == category]['Level 2'].unique()
    subcategories_per_category[category] = subcategories

subcategories_per_category

# Importação do dataset e criação do dataframe

In [7]:
# ENERGY & UTILITIES
other_utilities_descriptions = [#"Seeking eco-friendly water treatment chemicals for industrial wastewater plant—must meet regulations.",
                                #"Quotes needed for sustainable biofuels to power delivery vehicles—cost-effective options preferred.",
                                #"Want innovative solution for waste-to-energy conversion in facility.",
                                "Exploring alternative energy solutions to power our facilities, focusing on sustainable and eco-friendly options."]

eletrical_power_descriptions = [#"Need high-capacity transformers for expanding production facility—must meet reliability standards.",
                                #"Requesting proposals for solar panel installations on headquarters—harnessing solar energy for offices.",
                                #"Exploring backup power generators for continuous operations during outages—reliable and efficient.",
                                "Procuring reliable electrical power sources to meet the growing energy demands of our manufacturing units and office spaces."]


# HUMAN RESOURCERS
insurance_descriptions = [#"Health insurance coverage for employees, including medical, dental, and vision benefits.",
                          #"Requesting quotes for critical illness and disability insurance, prioritizing well-being and financial security.",
                          #"Exploring life insurance options for employees, aligning with our commitment to a secure work environment.",
                          "Securing comprehensive insurance coverage for our employees, prioritizing their well-being and financial security."]

talent_management_descriptions = [#"Looking for a talent management system with features like applicant tracking and performance evaluation.",
                                  #"Consulting services for talent development and retention, focusing on employee engagement.",
                                  #"Proposals for leadership training programs to groom our internal talent.",
                                  "Investing in talent management strategies to attract, develop, and retain skilled professionals for our expanding workforce."]

benefits_descriptions = [#"Exploring employee benefits packages, including health, wellness, and retirement benefits.",
                         #"Requesting quotes for flexible benefits plans to support work-life balance and employee well-being.",
                         #"Looking for innovative perks to create a positive workplace culture and enhance job satisfaction.",
                         "Enhancing employee benefits to promote a healthy work-life balance and foster a positive workplace culture."]

temporary_services_descriptions = [#"Temporary personnel services for short-term project peaks.",
                                   #"Proposals for temporary staffing solutions to cover employee absences.",
                                   #"Temporary personnel services for specialized projects.",
                                   "Engaging temporary personnel services to meet short-term staffing needs during peak business periods."]


# LOGISTICS
expedited_freight_descriptions = [
    #"Need expedited freight for time-sensitive shipments. Require reliable carriers for swift and on-time delivery.",
    #"Quotes for expedited freight to meet urgent deadlines. Need efficient and secure transportation.",
    #"Exploring options for expedited shipping to streamline supply chain. Interested in timely delivery track record.",
    "Optimizing our logistics network with expedited freight services for timely delivery of critical supplies and products."
]

freight_forwarding_descriptions = [
    #"Freight forwarding for international shipments. Need expertise in customs clearance and documentation.",
    #"Proposals for freight forwarding to optimize global logistics. Prefer comprehensive understanding of trade regulations.",
    #"Exploring freight forwarding for seamless transportation across borders. Interested in end-to-end visibility and tracking.",
    "Streamlining international shipping processes through efficient freight forwarding solutions for our global operations."
]

other_logistics_descriptions = [
    #"In need of specialized logistics for oversized or high-value items. Looking for providers experienced in unique cargo handling.",
    #"Requesting quotes for customized logistics to meet project requirements. Require flexibility and tailored solutions.",
    #"Exploring innovative logistics for unique business needs. Interested in providers offering creative and adaptable solutions.",
    "Improving overall logistics efficiency with innovative solutions beyond traditional freight and transportation services."
]

warehousing_distribution_descriptions = [
    #"Looking for warehousing and distribution to optimize inventory management. Need a partner with efficient order fulfillment.",
    #"Requesting proposals for warehousing to streamline supply chain. Prefer providers with advanced inventory tracking.",
    #"Exploring warehousing options for expanding product line. Interested in providers with nationwide distribution for quick fulfillment.",
    "Enhancing warehousing and distribution capabilities to ensure seamless storage and delivery of our products."
]


# MANUFACTURING
contract_manufacturing_descriptions = [
    #"Seeking a partner for electronic component manufacturing. Need precision engineering and quality assurance for technology products.",
    #"Contract manufacturing of customized machinery parts. Require a partner with a proven track record in high-quality components.",
    #"Contract manufacturing for pharmaceutical products. Need a certified facility with adherence to regulatory standards.",
    "Collaborating with reliable partners for contract manufacturing, ensuring high-quality production while focusing on core business activities."
    ]


# PROFESSIONAL SERVICES
financial_services_descriptions = [
    #"Financial advisory services to optimize our investment portfolio. Looking for expertise in risk management and financial planning.",
    #"Quotes for accounting and bookkeeping services. Require accurate financial reporting and compliance with tax regulations.",
    #"Exploring financial consulting for budget optimization. Need insights into cost-saving strategies and financial efficiency.",
    "Navigating financial strategies for sustainable growth and fiscal responsibility."
]

legal_services_descriptions = [
    #"Looking for legal services to handle contract reviews and negotiations. Require expertise in business law and contract compliance.",
    #"Seeking legal counsel for intellectual property protection. Need assistance with patent applications and trademark registrations.",
    #"In need of legal services for employment law compliance. Require guidance on HR policies and labor regulations.",
    "Securing legal expertise to ensure compliance and mitigate risks."
]

consulting_descriptions = [
    #"Seeking management consulting services for organizational strategy development. Need insights into improving operational efficiency.",
    #"In need of IT consulting for system integration. Require expertise in optimizing our technology infrastructure.",
    #"Exploring sustainability consulting for environmental initiatives. Need guidance on eco-friendly practices and corporate responsibility.",
    "Partnering with consultants for strategic insights and business optimization."
]

public_relations_descriptions = [
    #"Looking for public relations services to enhance our brand image. Require media outreach and strategic communication planning.",
    #"Seeking PR support for product launches and events. Need expertise in creating impactful press releases and media coverage.",
    #"In need of crisis management PR services. Require a strategic approach to handle potential reputational challenges.",
    "Crafting compelling narratives and maintaining positive public perception."
]

accounting_tax_descriptions = [
    #"Seeking accounting services for tax preparation and filing. Need expertise in compliance with current tax regulations.",
    #"In need of audit services to ensure financial transparency. Require a thorough examination of financial records.",
    #"Exploring outsourced payroll services. Need accurate and timely payroll processing.",
    "Ensuring financial accuracy and compliance through expert accounting and tax services."
]

outsourced_operations_description = [
    "Streamlining operations through strategic outsourcing partnerships."
]

audit_assurance_descriptions = [
    #"Seeking audit services for financial statement verification. Need assurance of accurate and reliable financial reporting.",
    #"In need of assurance services for data security compliance. Require a thorough examination of our cybersecurity measures.",
    #"Looking for sustainability assurance services. Need verification of our environmental sustainability initiatives.",
    "Maintaining transparency and credibility with thorough audit and assurance processes."
]

engineering_services_descriptions = [
    #"In need of engineering design services for product development. Require expertise in CAD modeling and prototyping.",
    #"Seeking structural engineering services for construction projects. Need precise calculations and adherence to safety standards.",
    #"Looking for environmental engineering services for sustainable initiatives. Require guidance on eco-friendly practices and compliance.",
    "Innovating product development through cutting-edge engineering solutions."
]

translation_localization_descriptions = [
    #"Looking for translation services for our global marketing materials. Require accurate and culturally sensitive translations.",
    #"Seeking localization services for our software products. Need adaptation for different languages and regions.",
    #"In need of translation services for legal documents. Require certified translations for international compliance.",
    "Expanding globally with accurate and culturally sensitive translation services."
]



# REAL STATE & FACILITIES
security_descriptions = [#"Requesting a comprehensive security audit for our office premises to enhance safety measures.",
                         #"Procuring surveillance cameras and access control systems to bolster the security infrastructure.",
                         #"In need of security personnel services for a corporate event hosted at our Real Estate property.",
                         "Ensuring a secure work environment with state-of-the-art security solutions."
]

food_beverage_descriptions = [#"Ordering catering services for an upcoming meeting at our Real Estate facility.",
                              #"Purchasing a variety of snacks and beverages to stock the pantry in our office space.",
                              #"Requesting quotes for a food and beverage vendor to serve employees during a team-building event.",
                              "Curating a delightful workplace experience with quality food and beverage offerings."]

construction_descriptions = [#"Seeking construction materials for renovating our Real Estate property's common areas.",
                             #"Initiating a construction project to expand our existing office space.",
                             #"Ordering construction equipment for a maintenance project within our owned Real Estate premises.",
                             "Building a sustainable future with eco-friendly construction practices."]

facilities_supplies_descriptions = [#"Restocking essential office supplies such as paper, ink, and stationery.",
                                    #"Procuring cleaning supplies and maintenance tools for our facilities team.",
                                    #"Ordering ergonomic furniture and accessories to improve workplace comfort.",
                                    "Sourcing reliable supplies to maintain a seamless and well-equipped workplace."]

facilities_services_descriptions = [#"Hiring external facilities management services for day-to-day operations.",
                                    #"Requesting HVAC maintenance services for our Real Estate property.",
                                    #"Seeking landscaping services to enhance the exterior of our office building.",
                                    "Outsourcing facility management to optimize day-to-day operations."]

office_supplies_descriptions = [#"Restocking office essentials including pens, notebooks, and printer cartridges.",
                                #"Ordering new office furniture and storage solutions for increased efficiency.",
                                #"Procuring whiteboards and presentation materials for our meeting rooms.",
                                "Ensuring a productive workspace with high-quality office supplies."]

real_estate_leased_descriptions = [#"Renewing the lease agreement for our corporate office space.",
                                   #"Negotiating terms for leasing additional square footage for expanding teams.",
                                   #"Exploring options for leasing a satellite office in a strategic location.",
                                   "Strategically leasing properties to support our expanding business."]

real_estate_owned_descriptions = [#"Initiating property maintenance for our owned Real Estate to ensure longevity.",
                                  #"Ordering landscaping services to enhance the curb appeal of our owned property.",
                                  #"Seeking professional appraisal services for our Real Estate assets.",
                                  "Investing in owned properties for long-term stability."]

transportation_descriptions = [#"Arranging transportation services for employees commuting to our Real Estate site.",
                               #"Requesting quotes for a corporate vehicle fleet for our Facilities team.",
                               #"Exploring options for shuttle services to improve employee transportation convenience.",
                               "Managing efficient transportation services for seamless business operations."]


# SALES, MARKETING & EVENTS
market_research_descriptions = [
    #"Conduct thorough market research to identify emerging trends and inform strategic decisions.",
    #"Analyze market dynamics to refine marketing strategies and optimize industry positioning.",
    #"Utilize advanced methodologies for anticipating changes and staying ahead of the competition.",
    "Staying ahead with data-driven market insights for strategic decision-making."
]

media_descriptions = [
    #"Develop multimedia campaigns to enhance brand visibility across digital, TV, and print.",
    #"Create engaging social media content for wider online reach and meaningful audience connection.",
    #"Secure media coverage through collaborations with influencers and industry leaders.",
    "Amplifying brand presence through targeted and engaging media campaigns."
]

commercial_payments_descriptions = [
    #"Implement efficient commercial payment solutions to streamline transactions.",
    #"Negotiate contracts with payment processors for competitive rates.",
    #"Enhance customer experience with a secure and user-friendly online payment system.",
    "Streamlining financial transactions for smooth business operations."
]

fullService_creativeAgency_descriptions = [
    #"Partner with a creative agency for a cohesive brand identity and design.",
    #"Collaborate on multimedia content creation for a captivating brand presence.",
    #"Execute innovative marketing campaigns for a standout brand in the market.",
    "Fueling creativity with comprehensive agency support for impactful campaigns."
]

sponsorships_contributions_descriptions = [
    #"Identify strategic sponsorships for brand awareness and community engagement.",
    #"Establish partnerships with organizations, leveraging sponsorships for brand image.",
    #"Support social causes through structured corporate contributions."
    "Building brand affinity through strategic sponsorships and community contributions."
]

events_descriptions = [
    #"Organize product launch events to generate buzz and attract stakeholders.",
    #"Coordinate regional roadshows for personal audience connection and feedback.",
    #"Execute thought leadership seminars for industry authority positioning.",
    "Creating memorable experiences with meticulously planned corporate events."
]

original_content_descriptions = [
    #"Develop a content strategy with original blog posts and articles for thought leadership.",
    #"Produce engaging videos showcasing behind-the-scenes aspects of the company.",
    #"Curate a branded podcast featuring industry experts for valuable insights.",
    "Engaging audiences through authentic and compelling original content."
]

production_descriptions = [
    #"Oversee end-to-end production of marketing materials for consistent branding.",
    #"Collaborate on visually appealing product videos to showcase offerings.",
    #"Manage logistics for large-scale promotional events, ensuring seamless execution.",
    "Executing seamless production processes for high-quality deliverables."
]

retail_programs_descriptions = [
    #"Implement targeted retail programs to enhance product visibility and drive sales.",
    #"Create exclusive in-store promotions to incentivize customer engagement.",
    #"Collaborate with retail partners on eye-catching displays for product highlights.",
    "Driving sales with innovative and effective retail programs."
]

other_marketing_descriptions = [
    #"Explore unconventional marketing channels and guerrilla tactics for brand differentiation.",
    #"Engage in cross-promotional partnerships for expanded reach and new demographics.",
    #"Implement experiential marketing campaigns for memorable brand experiences.",
    "Exploring diverse marketing services to meet unique business needs."
]

promotional_items_descriptions = [
    #"Design and distribute branded promotional items for a lasting connection.",
    #"Develop targeted promotional product campaigns to incentivize engagement.",
    #"Create limited-edition items for product launches, generating excitement.",
    "Enhancing brand visibility through thoughtfully selected promotional items."
]

messaging_telecom_descriptions = [
    #"Optimize messaging services for seamless communication with customers.",
    #"Explore innovative telecom services for enhanced customer outreach.",
    #"Implement a comprehensive messaging strategy for brand consistency.",
    "Ensuring seamless communication with reliable messaging and telecom solutions."
]


# TECHNOLOGY/TELECOM
software_support_descriptions = [
    #"Need advanced project management software with reliable customer support for streamlined collaboration.",
    #"Seeking software solutions and support to optimize data analytics for better decision-making.",
    #"Require cybersecurity software and expert support to fortify digital infrastructure.",
    "Leveraging cutting-edge software and support for enhanced digital capabilities."
]

technical_professional_descriptions = [
    #"Hiring technical professionals for seamless integration of new cloud solutions.",
    #"Need technical services for a network audit and optimization project.",
    #"Engaging technical services for an IT assessment and cybersecurity measures.",
    "Engaging technical professionals for specialized and efficient services."
]

computer_hardware_descriptions = [
    #"Upgrading workstations with high-performance CPUs and GPUs for design and development.",
    #"Procuring top-of-the-line computer hardware components for enhanced server capabilities.",
    #"In need of reliable desktop computers, monitors, and accessories.",
    "Optimizing performance with state-of-the-art computer hardware solutions."
]

office_equipment_descriptions = [
    #"Sourcing ergonomic office equipment for a comfortable work environment.",
    #"Upgrading office printers and copiers for improved document management.",
    #"Exploring comprehensive office technology solutions for enhanced productivity.",
    "Ensuring a productive workplace with advanced office equipment and services."
]

components_descriptions = [
    #"Seeking high-quality hardware components for precision in manufacturing.",
    #"Purchasing electronic components to support innovative product development.",
    #"Sourcing specialized components for R&D projects.",
    "Sourcing high-quality components for reliable technology solutions."
]

connectivity_bandwidth_descriptions = [
    #"Upgrading network infrastructure for higher bandwidth and seamless communication.",
    #"Exploring connectivity solutions for increased bandwidth to support cloud services.",
    #"Seeking reliable connectivity for remote workforce with a focus on speed and stability.",
    "Staying connected with robust connectivity and high bandwidth solutions."
]

network_hardware_descriptions = [
    #"Procuring advanced network hardware for data center expansion.",
    #"Upgrading networking equipment for optimized data transfer speeds.",
    #"In need of robust network hardware solutions for a secure corporate network.",
    "Building a secure and efficient network infrastructure with reliable hardware."
]

telecomEquipment_telecomServices_descriptions = [
    #"Acquiring state-of-the-art telecom equipment for improved communication infrastructure.",
    #"Exploring telecom solutions for seamless integration of voice and data services.",
    #"Seeking telecom equipment and services to enhance business continuity plans.",
    "Enhancing communication channels with cutting-edge telecom equipment and services."
]


# TRAVEL & EXPENSE
travel_agency_descriptions = [
    #"Seeking travel agency for efficient corporate travel bookings with cost-effective options.",
    #"In need of a reliable travel agency for group travel arrangements and event scheduling.",
    #"Looking for a travel agency specialized in international business travel with visa processing expertise.",
    "Simplifying business travel with the expertise of a reliable travel agency."
]

travel_food_beverage_descriptions = [
    #"In need of catering for corporate events with a diverse menu and dietary options.",
    #"Seeking a food and beverage supplier for office pantry with a variety of snacks and catering options.",
    #"Looking for a restaurant partner for corporate dining with a menu tailored to employees' tastes.",
    "Catering to diverse tastes and preferences with quality food and beverage options."
]

taxi_rideshare_descriptions = [
    #"Seeking a reliable rideshare service for safe and efficient employee transportation.",
    #"In need of taxi or limo services for executive travel with punctuality and professionalism.",
    #"Looking for a rideshare solution for airport transfers with convenience and cost-effectiveness.",
    "Facilitating convenient transportation with efficient taxi, limo, and rideshare services."
]

other_travel_descriptions = [
    #"Seeking miscellaneous travel-related services for unique employee perks and rewards.",
    #"In need of travel insurance for business trips with comprehensive coverage.",
    #"Looking for a service to manage corporate travel expenses efficiently.",
    "Exploring unique travel solutions to cater to varied business needs."
]


In [8]:
# Criar um dicionário com as listas
data = {
    'Descriptions':
    other_utilities_descriptions + eletrical_power_descriptions + insurance_descriptions + talent_management_descriptions + benefits_descriptions +
    temporary_services_descriptions + expedited_freight_descriptions + freight_forwarding_descriptions + other_logistics_descriptions +
    warehousing_distribution_descriptions + contract_manufacturing_descriptions + financial_services_descriptions + legal_services_descriptions +
    consulting_descriptions + public_relations_descriptions + accounting_tax_descriptions + outsourced_operations_description + audit_assurance_descriptions +
    engineering_services_descriptions + translation_localization_descriptions + security_descriptions + food_beverage_descriptions +
    construction_descriptions + facilities_supplies_descriptions + facilities_services_descriptions + office_supplies_descriptions +
    real_estate_leased_descriptions + real_estate_owned_descriptions + transportation_descriptions + market_research_descriptions + media_descriptions +
    commercial_payments_descriptions + fullService_creativeAgency_descriptions + sponsorships_contributions_descriptions + events_descriptions +
    original_content_descriptions + production_descriptions + retail_programs_descriptions + other_marketing_descriptions + promotional_items_descriptions +
    messaging_telecom_descriptions + software_support_descriptions + technical_professional_descriptions + computer_hardware_descriptions +
    office_equipment_descriptions + components_descriptions + connectivity_bandwidth_descriptions + network_hardware_descriptions +
    telecomEquipment_telecomServices_descriptions + travel_agency_descriptions + travel_food_beverage_descriptions +  taxi_rideshare_descriptions +
    other_travel_descriptions
}

df_descriptions = pd.DataFrame(data)

In [9]:
df_level_2 = pd.DataFrame({
    'Level 2': ['Other Utilities'] + ['Electrical Power'] + ['Insurance'] + ['Talent Management'] + ['Benefits'] + ['Temporary Personnel Services'] +
    ['Expedited Freight'] + ['Freight Forwarding'] +  ['Other Logistics'] + ['Warehousing & Distribution'] + ['Contract Manufacturing'] + ['Financial Services'] +
    ['Legal Services'] +  ['Consulting'] + ['Public Relations'] +  ['Accounting and Tax Services'] + ['Outsourced Operations'] + ['Audit & Assurance'] +
    ['Engineering Services'] +  ['Translation & Localization Services'] + ['Security'] + ['Food & Beverage'] +  ['Construction'] + ['Facilities Supplies'] +
    ['Facilities Services'] + ['Office Supplies'] + ['Real Estate Leased'] + ['Real Estate Owned'] + ['Transportation'] + ['Market Research'] + ['Media'] +
    ['Commercial Payments'] + ['Full Service Creative Agency'] + ['Sponsorships and Contributions'] + ['Events'] + ['Original Content'] + ['Production'] + ['Retail Programs'] +
    ['Other Marketing Services'] +  ['Promotional Items'] + ['Messaging and Telecom Services'] + ['Software & Support'] + ['Technical Professional Services'] +
    ['Computer Hardware'] + ['Office Equipment & Services'] + [ 'Components'] + ['Connectivity & Bandwidth'] + ['Network Hardware'] + ['Telecom Equipment & Telecom Services'] +
    ['Travel Agency'] + ['Food & Beverage'] + ['Taxi/Limo/Rideshare'] + ['Other Travel']

})

In [10]:
df_descriptions2 = pd.concat([df_descriptions, df_level_2], axis=1)

In [11]:
df_descriptions2

,Descriptions,Level 2
0,Exploring alternative energy solutions to powe...,Other Utilities
1,Procuring reliable electrical power sources to...,Electrical Power
2,Securing comprehensive insurance coverage for ...,Insurance
3,Investing in talent management strategies to a...,Talent Management
4,Enhancing employee benefits to promote a healt...,Benefits
5,Engaging temporary personnel services to meet ...,Temporary Personnel Services
6,Optimizing our logistics network with expedite...,Expedited Freight
7,Streamlining international shipping processes ...,Freight Forwarding
8,Improving overall logistics efficiency with in...,Other Logistics
9,Enhancing warehousing and distribution capabil...,Warehousing & Distribution


# Tratamento dos dados

## Pré-processamento do PLN

A função "preprocess_text()" tem o objetivo de aplicar algumas técnicas de pré-processamento do PLN em uma string que receber como parâmetro

In [12]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Definindo método de pré-processamento de texto
def preprocess_text(text):

    tokens = tokenize.word_tokenize(text, language='english')  # Tokenização
    tokens = [word for word in tokens if word.isalnum()]  # Removendo caracteres não alfanuméricos
    tokens = [word.lower() for word in tokens]  # Convertendo para minúsculas
    # print("Tokenização: ", tokens)

    tokens = [word for word in tokens if word not in stop_words]  # Removendo Stopwords
    # print("Remoção das Stopwords: ", tokens)

    tokens = [stemmer.stem(word) for word in tokens]  # Aplicando stemming
    # print("Aplicação do Stemming: ", stemmed_tokens)

    return tokens

In [13]:
df_tokenize = df_descriptions2.copy()

In [14]:
df_tokenize['Descriptions'] = df_tokenize['Descriptions'].apply(preprocess_text) # aplicação da função acima em todas as linhas da coluna

In [15]:
df_tokenize

,Descriptions,Level 2
0,"[explor, altern, energi, solut, power, facil, ...",Other Utilities
1,"[procur, reliabl, electr, power, sourc, meet, ...",Electrical Power
2,"[secur, comprehens, insur, coverag, employe, p...",Insurance
3,"[invest, talent, manag, strategi, attract, dev...",Talent Management
4,"[enhanc, employe, benefit, promot, healthi, ba...",Benefits
5,"[engag, temporari, personnel, servic, meet, st...",Temporary Personnel Services
6,"[optim, logist, network, expedit, freight, ser...",Expedited Freight
7,"[streamlin, intern, ship, process, effici, fre...",Freight Forwarding
8,"[improv, overal, logist, effici, innov, solut,...",Other Logistics
9,"[enhanc, wareh, distribut, capabl, ensur, seam...",Warehousing & Distribution


## Vetorização

O processo de vetorização foi feito utilizando o Word2Vec, e sua função está definida abaixo. O Word2Vec é uma técnica de aprendizado de máquina que converte palavras em vetores numéricos contínuos, permitindo que modelos capturem relações semânticas e sintáticas entre palavras com base em seus contextos de ocorrência. Tal processo ofi aplicado apenas na colunas "GL Description", ou seja, a colunas com os dados de entrada.

In [16]:
df_transformers = df_tokenize['Descriptions'].to_frame()
df_transformers

,Descriptions
0,"[explor, altern, energi, solut, power, facil, ..."
1,"[procur, reliabl, electr, power, sourc, meet, ..."
2,"[secur, comprehens, insur, coverag, employe, p..."
3,"[invest, talent, manag, strategi, attract, dev..."
4,"[enhanc, employe, benefit, promot, healthi, ba..."
5,"[engag, temporari, personnel, servic, meet, st..."
6,"[optim, logist, network, expedit, freight, ser..."
7,"[streamlin, intern, ship, process, effici, fre..."
8,"[improv, overal, logist, effici, innov, solut,..."
9,"[enhanc, wareh, distribut, capabl, ensur, seam..."


In [17]:
def Vetorizer(dataframe,model):

  desc_column = dataframe['Descriptions']
  # Calcula os embeddings para a coluna de texto

  embeddings_liz = model.encode(desc_column.tolist())
  # Cria um novo DataFrame com os embeddings
  embeddings_df = pd.DataFrame(embeddings_liz)

  return embeddings_df

In [19]:
vetorizadorTransformers = SentenceTransformer('distiluse-base-multilingual-cased')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [20]:
df_transformers = Vetorizer(df_transformers, vetorizadorTransformers)
df_transformers

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.020738,0.019115,0.002267,0.005506,0.001466,0.015851,0.023621,-0.069516,0.010350,-0.014393,...,0.021196,-0.006271,-0.009913,-0.004183,-0.044319,-0.088991,-0.015413,0.022465,-0.031122,-0.001280
1,-0.009279,0.013067,-0.011499,-0.005483,-0.004331,-0.011143,0.009267,-0.029440,-0.027048,-0.017423,...,0.008586,-0.044128,-0.001408,0.012201,-0.017421,-0.043290,0.005564,-0.021332,-0.001158,-0.017780
2,-0.015136,-0.024918,-0.033574,0.009433,0.013207,-0.005879,-0.004073,-0.065764,-0.030756,0.018456,...,0.005753,-0.007808,-0.012939,0.006060,-0.007889,-0.047644,-0.000912,-0.017562,0.000263,-0.039586
3,0.009296,-0.035058,-0.032492,0.031013,0.004221,0.021084,-0.026102,-0.010985,-0.029324,-0.003231,...,0.010493,-0.051674,0.020580,0.040971,-0.055424,-0.090467,0.056452,-0.025203,0.007434,0.031318
4,0.038843,0.051459,-0.009847,0.004235,-0.004382,-0.012234,-0.037556,-0.034058,-0.049940,-0.016429,...,0.016376,-0.034286,0.002451,0.015938,-0.021266,-0.039720,-0.032119,-0.046512,0.004106,-0.033494
5,0.017418,0.010661,-0.026928,-0.021658,-0.005481,-0.013997,-0.023010,-0.011365,-0.028050,-0.006279,...,0.084963,-0.024073,0.024855,0.005798,0.004883,-0.045924,0.022333,-0.003092,-0.017452,-0.019887
6,-0.027689,-0.007754,-0.026712,-0.002577,-0.006078,0.031948,0.000644,-0.047421,-0.005208,-0.013961,...,-0.002259,0.011360,-0.027440,0.061499,0.002307,-0.069624,-0.030811,0.047353,0.025810,-0.040320
7,-0.040576,0.024209,-0.043468,-0.025142,-0.042030,-0.016127,-0.004060,-0.047366,-0.028992,0.001929,...,0.037136,-0.035392,-0.041708,0.028579,-0.023429,0.023234,-0.018447,0.013992,-0.012373,-0.061202
8,-0.017657,-0.021993,-0.029405,-0.011828,0.014593,0.004275,-0.019045,-0.006057,-0.032270,-0.024676,...,0.022552,-0.002826,-0.033914,0.035319,-0.005685,-0.015272,0.013627,-0.013736,-0.033608,-0.002305
9,-0.003712,0.049899,-0.001753,-0.014865,-0.021349,-0.021942,0.006070,-0.044276,-0.030453,-0.010098,...,0.018433,0.012941,-0.020458,-0.004665,-0.016031,-0.020210,0.002281,-0.055831,-0.003670,-0.037771


## Similaridade de cosseno

In [22]:
new_description = ["Managing the production process for marketing materials and promotional content."]
df_test = pd.DataFrame(new_description, columns=['Descriptions'])
df_test['Descriptions'] = df_test['Descriptions'].apply(preprocess_text)
df_new_description2 = Vetorizer(df_test, vetorizadorTransformers)

In [23]:
df_new_description2

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.022123,0.042915,-0.023342,-0.007388,-0.033036,-0.00509,0.000829,-0.003039,-0.061855,-0.02559,...,0.00826,0.007108,0.007926,0.007104,-0.012019,-0.022487,0.014052,0.007455,0.005432,-0.023682


In [24]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity_v2(vectors, phrase_vector):
    # Convertendo phrase_vector para um formato compatível, se necessário
    if isinstance(phrase_vector, pd.Series):
        phrase_vector = phrase_vector.values.reshape(1, -1)

    # Convertendo o DataFrame 'vectors' em uma matriz NumPy, se necessário
    if isinstance(vectors, pd.DataFrame):
        vectors = vectors.values

    # Calculando a similaridade do cosseno
    similarity_array = cosine_similarity(vectors, phrase_vector)

    # Retornando a similaridade como uma lista de valores
    return similarity_array.flatten().tolist()


In [25]:
similarity = calculate_similarity_v2(df_transformers, df_new_description2)
similarity

[0.355470210313797,
 0.47844040393829346,
 0.3581234812736511,
 0.3636937439441681,
 0.48779380321502686,
 0.29332810640335083,
 0.33488181233406067,
 0.38595694303512573,
 0.41665202379226685,
 0.49838268756866455,
 0.32952675223350525,
 0.35599666833877563,
 0.39289480447769165,
 0.29245907068252563,
 0.5324954390525818,
 0.380186527967453,
 0.4519445300102234,
 0.3484441637992859,
 0.7687146663665771,
 0.30388879776000977,
 0.561933159828186,
 0.5165806412696838,
 0.3666653037071228,
 0.4623252749443054,
 0.4463335871696472,
 0.945343554019928,
 0.43776530027389526,
 0.3746306002140045,
 0.47458702325820923,
 0.32790815830230713,
 0.44539397954940796,
 0.355842649936676,
 0.44170644879341125,
 0.4883391857147217,
 0.48052364587783813,
 0.48383253812789917,
 0.33042532205581665,
 0.42060694098472595,
 0.341322660446167,
 0.4684417247772217,
 0.3865426182746887,
 0.4142293334007263,
 0.38268256187438965,
 0.3994694948196411,
 0.9453434944152832,
 0.4964996874332428,
 0.327328860759735

In [26]:
def encontrar_max(array):
    if not array:
        return "Array vazio"

    maior_numero = array[0]
    index = 0

    for i in range(1, len(array)):
        if array[i] > maior_numero:
            maior_numero = array[i]
            index = i

    return maior_numero, index

In [29]:
a2, b2 = encontrar_max(similarity)
print(a2)
print(b2)

classification_level_2 = df_descriptions2['Level 2'].loc[b2 - 1]

if a2 <= 0.55:
  print('Me desculpe, mas eu não consigo te dar uma certeza da classificação desse pedido')

elif a2 >= 0.56 and a2 <= 0.64:
  print(f'Não tenho muita certeza sobre a classificação, acho que seja {classification_level_2}, mas verifique se você concorda')

elif a2 >= 0.65 and a2 <= 0.79:
  print(f'Não tenho muita certeza sobre a classificação, mas acredito que seja {classification_level_2}')

elif a2 >= 0.80:
  print(f'Olha, tenho quase certeza que a classificação para o nível 2 é {classification_level_2}')


0.945343554019928
25
Olha, tenho quase certeza que a classificação para o nível 2 é Facilities Services
